In [23]:
import torch
from torch import nn

In [ ]:
class Transformer_LSTM(nn.Module):
    def __init__(self,dimention,factor_num, sequence, fully_connect_layer_neural):
        super(Transformer_LSTM, self).__init__()
        self.factor_num = factor_num  # 108
        self.sequence = sequence # 
        self.dimention = dimention # 
        self.fc2_neuron = fully_connect_layer_neural  # 32

        # Layer
        self.dropout = nn.Dropout(0.2)
        self.LeakyReLU = nn.LeakyReLU()
        self.out = nn.Linear(self.fc2_neuron*2, 1)

    def forward(self, x):
        x = self.norm(x)
        x = torch.transpose(x, 0,1)  # x.shape: torch.Size([3, 512, 20, 108])
        
        start, (hn, cn) = self.transformer_lstm(x[0],2) # torch.Size([512, 20, 128])
        for i in range(1,x.shape[0]):
            add = self.transformer_lstm(x[i],2)
            start += add
            start = self.norm(start)
        # start = torch.Size([512, 20, 128])
        x, _ = self.attention_net(start, hn)
        x = self.norm(x)
        x = self.LeakyReLU(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred
    def norm(self,x):
        batch_num ,original_shape= x.shape[0] , x.shape
        x = x.reshape(batch_num, -1).float()
        x = torch.nn.BatchNorm1d(x.shape[-1])(x)
        x = x.reshape(original_shape)
        return x
    def transformer_lstm(self,x,layer_num):
        q = nn.Linear(self.factor_num, self.factor_num)(x)
        k = nn.Linear(self.factor_num, self.factor_num)(x)
        v = nn.Linear(self.factor_num, self.factor_num)(x)
        x, x_weight = nn.MultiheadAttention(self.factor_num, layer_num,batch_first=True)(q,k,v) # attn_output = torch.Size([512, 20, 128])
        # out = torch.Size([512, 20, 128])
        out, (hn,cn) = torch.nn.LSTM(self.factor_num, self.fc2_neuron, layer_num, batch_first=True, bidirectional=True, dropout=0.2)(x)
        return out, (hn,cn)
    def attention_net(self, lstm_output, final_state):
        # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
        # final_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        hidden = torch.cat((final_state[0], final_state[1]), dim=1).unsqueeze(
            2)  # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2)  # [batch_size,sequence]
        attn_weights = torch.nn.functional.softmax(attn_weights, 1)  # [batch_size,sequence]   # torch.Size([512, 20])
        # context: [batch_size, n_hidden * num_directions(=2)]
        output = torch.bmm(lstm_output.transpose(1, 2), attn_weights.unsqueeze(2)).squeeze(
            2)  # [batch_size, n_hidden * num_directions(=2)]
        return output, attn_weights

In [97]:
import os
from os import walk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torch.autograd import Variable
from progressbar import ProgressBar
from tqdm import tqdm
import time
import multiprocessing as mp
trainx = torch.randn(10000,3,20,108)
# trainx = torch.randn(10000,20,108)
trainy = torch.randn(10000,1)
print("trainx.shape: " , trainx.shape)

trainx.shape:  torch.Size([10000, 3, 20, 108])


In [98]:
train_dataset = Data.TensorDataset(trainx, trainy)
batch_size = 1024
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=16,
    pin_memory=True
)

In [133]:
class Transformer_LSTM(nn.Module):
    def __init__(self,dimention,factor_num, sequence, fully_connect_layer_neural,layer_num=2):
        super(Transformer_LSTM, self).__init__()
        self.factor_num = factor_num  # 108
        self.sequence = sequence # 
        self.dimention = dimention # 
        self.fc2_neuron = fully_connect_layer_neural  # 32
        
        # Layer
        self.bn1 = torch.nn.BatchNorm1d(self.dimention * self.factor_num* self.sequence)
        self.bn2 = torch.nn.BatchNorm1d(self.fc2_neuron*2* self.sequence)
        self.bn3 = torch.nn.BatchNorm1d(self.fc2_neuron*2)
        self.q_metrix = nn.Linear(self.factor_num, self.factor_num)
        self.k_metrix = nn.Linear(self.factor_num, self.factor_num)
        self.v_metrix = nn.Linear(self.factor_num, self.factor_num)
        self.MultiheadAttention = nn.MultiheadAttention(self.factor_num,layer_num,batch_first=True)
        self.lstm = nn.LSTM(self.factor_num, self.fc2_neuron, layer_num, batch_first=True, bidirectional=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.LeakyReLU = nn.LeakyReLU()
        self.out = nn.Linear(self.fc2_neuron*2, 1)
    def forward(self, x):
        x = self.norm_1(x)
        x = torch.transpose(x, 0,1)  # x.shape: torch.Size([3, 512, 20, 108])
        
        final, (hn, cn) = self.transformer_lstm(x[0],2) # torch.Size([512, 20, 128])
        for i in range(1,x.shape[0]):
            add , _ = self.transformer_lstm(x[i],2)
            final = self.skip_connection(final,add)
        # start = torch.Size([512, 20, 128])
        
        x, _ = self.attention_net(final, hn)
        x = self.bn3(x)
        x = self.LeakyReLU(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred
    def norm_1(self,x):
        batch_num ,original_shape= x.shape[0] , x.shape
        x = x.reshape(batch_num, -1)
        x = self.bn1(x)
        x = x.reshape(original_shape)
        return x
    def norm_2(self,x):
        batch_num ,original_shape= x.shape[0] , x.shape
        x = x.reshape(batch_num, -1)
        x = self.bn2(x)
        x = x.reshape(original_shape)
        return x
    def transformer_lstm(self,x,layer_num):
        q = self.q_metrix(x)
        k = self.k_metrix(x)
        v = self.v_metrix(x)
        x, x_weight = self.MultiheadAttention(q,k,v) # attn_output = torch.Size([512, 20, 128])
        # out = torch.Size([512, 20, 128])
        out, (hn,cn) = self.lstm(x)
        return out, (hn,cn)
    def skip_connection(self,origin,add):
        return self.norm_2(origin + add)
        
    def attention_net(self, lstm_output, final_state):
        # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
        # final_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        hidden = torch.cat((final_state[0], final_state[1]), dim=1).unsqueeze(
            2)  # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2)  # [batch_size,sequence]
        attn_weights = torch.nn.functional.softmax(attn_weights, 1)  # [batch_size,sequence]   # torch.Size([512, 20])
        # context: [batch_size, n_hidden * num_directions(=2)]
        output = torch.bmm(lstm_output.transpose(1, 2), attn_weights.unsqueeze(2)).squeeze(
            2)  # [batch_size, n_hidden * num_directions(=2)]
        return output, attn_weights

In [134]:
alphanet = Transformer_LSTM(3,108,20,64,"cuda:0")
# alphanet = AlphaNet_LSTM_V1(108,20,64)
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.01
loss_function = nn.MSELoss().cuda()
optimizer = optim.Adam(alphanet.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma = 0.5)
epoch_num = 50
loss_list = []

min_loss = float("inf")
for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad() # noticed:  the grad return to zero before starting the loop
        
        # forward + backward +update
        pred = alphanet(inputs.cuda())
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        with torch.autograd.detect_anomaly():
            loss.backward()
#         loss.backward()
        optimizer.step()
        
#         lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)
    loss_list.append(total_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

Transformer_LSTM(
  (bn1): BatchNorm1d(6480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(2560, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (q_metrix): Linear(in_features=108, out_features=108, bias=True)
  (k_metrix): Linear(in_features=108, out_features=108, bias=True)
  (v_metrix): Linear(in_features=108, out_features=108, bias=True)
  (MultiheadAttention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=108, out_features=108, bias=True)
  )
  (lstm): LSTM(108, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (LeakyReLU): LeakyReLU(negative_slope=0.01)
  (out): Linear(in_features=128, out_features=1, bias=True)
)


/home/wuwenjun/venv-jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  2%|▏         | 1/50 [00:03<03:05,  3.79s/it]

Epoch:  1  loss:  1.1999158142089843


  4%|▍         | 2/50 [00:08<03:17,  4.12s/it]

...

Epoch:  48  loss:  1.0376955627441407


 98%|█████████▊| 49/50 [03:36<00:04,  4.46s/it]

Epoch:  49  loss:  1.0382608215332032


100%|██████████| 50/50 [03:40<00:00,  4.42s/it]

Epoch:  50  loss:  1.0372375122070312


In [109]:
inputs.device

device(type='cuda', index=0)

In [82]:
a = a.reshape(3,-1)
a 

tensor([[ 1.8269e-01, -5.0047e-01,  3.3323e-02,  8.6941e-01],
        [ 3.2009e-02,  7.1281e-04,  1.2799e+00, -3.1223e-01],
        [-1.9265e-01,  8.5219e-01,  4.5535e-01, -1.4224e+00]])

In [69]:
def transformer_lstm(x,layer_num):
    q = nn.Linear(factor_num, factor_num)(x)
    k = nn.Linear(factor_num, factor_num)(x)
    v = nn.Linear(factor_num, factor_num)(x)
    x, x_weight = nn.MultiheadAttention(factor_num, layer_num,batch_first=True)(q,k,v) # attn_output = torch.Size([512, 20, 128])
    # out = torch.Size([512, 20, 128])
    out, (hn,cn) = torch.nn.LSTM(factor_num, 64, layer_num, batch_first=True, bidirectional=True, dropout=0.2)(x)
    return out

In [73]:
start = transformer_lstm(x[0],2) # torch.Size([512, 20, 128])
for i in range(1,x.shape[0]):
    add = transformer_lstm(x[i],2)
    start += add

In [77]:
start.shape

torch.Size([512, 20, 128])

In [72]:
start.shape

torch.Size([512, 20, 128])

In [62]:
batch_num ,original_shape= x.shape[0] , x.shape
x = x.reshape(batch_num, -1).float()
# x = torch.nn.BatchNorm1d(x.shape[-1])(x)
x = x.reshape(original_shape)

In [63]:
x

tensor([[[[ 0.3720, -0.8041, -0.9032, -0.6947],
          [ 0.0581, -0.5880,  0.8210, -0.5347],
          [ 1.5516, -2.1451,  1.2694, -0.2031]],

         [[-0.4983, -0.3954,  0.1497, -0.3727],
          [ 2.2113, -1.0530, -0.4277,  0.0852],
          [ 0.7044, -2.0854, -1.0672,  0.2603]],

         [[-2.0598, -0.3966,  1.0161,  0.4361],
          [-0.8015, -0.6283, -2.2997,  1.4656],
          [ 0.0649, -0.5740,  0.7617, -0.2757]]],


        [[[-0.5442, -0.9948,  0.8970, -0.2814],
          [-0.8861,  0.0641,  0.1193, -0.4461],
          [ 0.0212, -0.2006, -0.3645,  0.6403]],

         [[-1.7014, -0.5908,  0.8032, -0.5609],
          [-0.4263,  0.0032, -0.8050, -1.1229],
          [-0.6438,  0.5535,  0.0984,  1.1140]],

         [[ 1.1569,  1.1844, -0.9598, -1.7448],
          [-0.2499,  0.2846, -0.1533, -1.4458],
          [ 0.2298, -2.0872,  0.1569,  1.0752]]]])

In [60]:
original_shape

torch.Size([512, 20, 108])

In [53]:
x, x_weight = nn.MultiheadAttention(factor_num, 2,batch_first=True)(q,k,v)

In [20]:
out, (hn,cn) = torch.nn.LSTM(108, 64, 2, batch_first=True, bidirectional=True, dropout=0.2)(a)

In [34]:
multihead_attn = nn.MultiheadAttention(128, 4,batch_first=True)
attn_output, attn_output_weights = multihead_attn(out,out,out)

In [35]:
out.shape

torch.Size([512, 20, 128])

In [41]:
nn.Linear(128, 128)(out).shape

torch.Size([512, 20, 128])

In [37]:
attn_output_weights.shape

torch.Size([512, 20, 20])

In [17]:
a.reshape(3,4,2)

tensor([[[ 0.5335, -1.8345],
         [-0.4648,  0.3510],
         [-0.8114, -0.6642],
         [-1.6805,  0.6834]],

        [[ 0.9820, -1.5513],
         [-2.2197,  0.6072],
         [ 1.0861,  0.8014],
         [ 0.6801,  0.5972]],

        [[-0.9948, -0.0429],
         [-0.0649, -0.1456],
         [-0.1955,  1.2058],
         [ 0.0729, -1.0356]]])